<a href="https://colab.research.google.com/github/BenstaaliKamel/BABot_FAQ/blob/main/SmartCityAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Commençons

- Un environnement virtuel est activé pour ce notebook
- Impression de la liste des dépendances installées dans cet environnement virtuel dans un fichier .txt

In [ ]:
#!pip freeze > requirements.txt

In [ ]:
import pandas as pd

### Chargement du jeu de données

In [ ]:
data_tree=pd.read_csv('https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+2+Participez+%C3%A0+un+concours+sur+la+Smart+City/p2-arbres-fr.csv', sep=';')

In [ ]:
data_tree.head()

In [ ]:
data_tree.shape

En utilisant cette fonction précédente, nous voyons que notre ensemble de données contient des informations sur 200137 arbres avec 18 champs,

Identifions ces colonnes:

# Analyses naïves

## Première vue des variables

In [ ]:
data_tree.columns

data_tree

In [ ]:
data_tree.dtypes

In [ ]:
data_tree.info()

Ce que l'on remarque:

De ce résumé sur les colonnes, on voit qu'il y a une colonne qui ne contient que des valeurs nulles "numero"

## Valeurs manquantes

In [ ]:
data_tree.isnull().sum()

D'autres colonnes ont un pourcentage important de valeurs nulles, cependant nous nous occuperons des valeurs nulles et d'autres types d'erreurs plus tard car nous souhaitons supprimer certaines colonnes inutiles pour notre analyse.

# Choix des données

## Filtrage de notre ensemble de données

Nous souhaitons maintenant utiliser uniquement les champs dont nous avons besoin pour notre analyse. Nous devons donc en supprimer un grand nombre:

In [ ]:
# Now we want to delete certain useless columns for our analysis
data_tree = data_tree.drop(['id_emplacement', 'type_emplacement', 'complement_addresse' , 'numero', 'lieu', 'libelle_francais', 'variete'], axis=1)

In [ ]:
data_tree.head(10)

## Les champs nécessaires :

In [ ]:
data_tree.columns

In [ ]:
data_tree.info()

In [ ]:
data_tree.describe(include='all')

In [ ]:
#Je veux mettre la colonne "id_emplacement" en premiere position:
new_order = ['id'] + [col for col in data_tree.columns if col != 'id']
data_tree = data_tree[new_order]
new_order = [col for col in data_tree.columns if col != 'remarquable'] + ['remarquable']
data_tree = data_tree[new_order]

In [ ]:
data_tree.head(10)

## Prétraitement nécéssaire

convertissons les valeurs de la colonne « hauteur » en cm pour correspondre à l'unité de la colonne « circonférence »

In [ ]:
data_tree['hauteur_cm'] = data_tree['hauteur_m'] * 100

new_order = ['id','domanialite','arrondissement','genre','espece','circonference_cm','hauteur_cm','stade_developpement','geo_point_2d_a','geo_point_2d_b','remarquable']
data_tree = data_tree[new_order]
data_tree

In [ ]:
data_tree.info()

# Analyse univariée
Avant de procèder à cette étape nous devons faire quelques

## Erreurs syntaxiques
Fonctions pour vérifier la présence de valeurs non conformes au type souhaité

In [ ]:
import numpy as np
def replace_non_type_with_null(df, column_name, data_type):
    """
    Remplace les valeurs d'une colonne spécifiée par NaN si elles ne correspondent pas au type de données spécifié.

    Paramètres :
    df (pandas.DataFrame) : Le DataFrame d'entrée.
    column_name (str) : Le nom de la colonne à traiter.
    data_type (type) : Le type de données souhaité (str, int ou float).

    Retourne :
    pandas.DataFrame : Le DataFrame avec la colonne spécifiée traitée.
    """
    if data_type not in [str, int, float]:
        raise ValueError("data_type must be str, int, or float")

    if data_type == str:
        df[column_name] = df[column_name].apply(lambda x: x if isinstance(x, (str, object)) else np.nan)
    elif data_type == int:
        df[column_name] = df[column_name].apply(lambda x: x if isinstance(x, int) else np.nan)
    elif data_type == float:
        df[column_name] = df[column_name].apply(lambda x: x if isinstance(x, float) else np.nan)

    return df

In [ ]:
replace_non_type_with_null(data_tree, "domanialite", str)
replace_non_type_with_null(data_tree, "arrondissement", str)
replace_non_type_with_null(data_tree, "genre", str)
replace_non_type_with_null(data_tree, "espece", str)
replace_non_type_with_null(data_tree, "stade_developpement", str)
replace_non_type_with_null(data_tree,'id', int)
replace_non_type_with_null(data_tree,'circonference_cm', int)
replace_non_type_with_null(data_tree,'hauteur_cm', int)
replace_non_type_with_null(data_tree,'geo_point_2d_a', float)
replace_non_type_with_null(data_tree,'geo_point_2d_b', float)
replace_non_type_with_null(data_tree,'remarquable', float)
data_tree.info()

In [ ]:
data_tree

In [ ]:

data_tree.describe(include='all')

## Doublons

l'utilisation de la méthode  .duplicated()  permet de vérifier si au sein d'une ou plusieurs variables, il existe des doublons.
Donc on crée un fonction qui verifie des possibles doublons dans les combinaisons 'id' et les points de localisation

In [ ]:
def verifier_doublons(df, subset=None, keep='first'):
    # Identifier les doublons
    doublons = df[df.duplicated(subset=subset, keep=keep)]

    # Compter le nombre de doublons
    nombre_doublons = len(doublons)

    # Afficher les résultats
    print(f"Nombre total de lignes : {len(df)}")
    print(f"Nombre de lignes uniques : {len(df.drop_duplicates(subset=subset, keep=keep))}")
    print(f"Nombre de lignes dupliquées : {nombre_doublons}")

    # Afficher les doublons si présents
    if nombre_doublons > 0:
        print("\nLignes dupliquées :")
        print(doublons)
    return doublons
verifier_doublons(data_tree, ["id", "geo_point_2d_a", "geo_point_2d_b"])#ajouter id-emplacement

Vérifications des ratios de valeurs nulles

In [ ]:
data_tree.isna().mean()

## Statistiques univariées

In [ ]:
def display_univariate_stats(df):
    """
    Affiche différentes statistiques univariées pour chaque colonne d'un DataFrame.
    Si la colonne est de type numérique, les stats à afficher sont:
        -Statistiques descriptives (count, mean, std, min, 25%, 50%, 75%, max)
        -L'asymétrie (skewness) et l'aplatissement (kurtosis)
    Si la colonne est de type catégorique, les stats à afficher sont:
        -Fréquences
        -Pourcentages
        -Mode
    Args:
    df (pandas.DataFrame): Le DataFrame à analyser
    """
    for column in df.columns:
        print(f"\n{'='*50}")
        print(f"Analyse de la colonne: {column}")
        print(f"{'='*50}")

        # Déterminer le type de la colonne
        if pd.api.types.is_numeric_dtype(df[column]):
            print("Type: Numérique")

            # Statistiques descriptives
            desc = df[column].describe()
            df[column].median()
            print("\nStatistiques descriptives:")
            print(desc)
            print("mediane:  ",df[column].median())

            # Asymétrie et aplatissement
            print(f"\nAsymétrie (Skewness): {df[column].skew():.2f}")
            print(f"Aplatissement (Kurtosis): {df[column].kurtosis():.2f}")

        elif pd.api.types.is_object_dtype(df[column]) or pd.api.types.is_categorical_dtype(df[column]):
            print("Type: Catégoriel")

            # Fréquences
            value_counts = df[column].value_counts()
            print("\nFréquences:")
            print(value_counts)

            # Pourcentages
            percentages = df[column].value_counts(normalize=True) * 100
            print("\nPourcentages:")
            print(percentages)

            # Mode
            print(f"\nMode: {df[column].mode()[0]}")


        else:
            print("Type: Inconnu ou non pris en charge")
            print(df[column].dtype)

    print("\nAnalyse terminée.")

In [ ]:
display_univariate_stats(data_tree)

## Catégorisation des variables

Pour les varibles qualitatives, on va regrouper certaines valeurs ( les moins fréquentes ) en une seule valeur 'autres' pour pouvoir faire des représentations graphiques plus claires, ce traitement va concerner les colonnes: 'domanialite', 'arrondissement', 'genre', 'espece'.

In [ ]:
import numpy as np
import pandas as pd

def convert_to_top_5_and_others(df, column_name, top_n=5, others_label='Autres'):
    """
    Convertit les valeurs les moins fréquentes d'une colonne qualitative en 'Autres',
    ne gardant que les valeurs qui font partie des n valeurs les plus fréquentes.
    Les valeurs nulles sont conservées telles quelles.

    Args:
    df (pandas.DataFrame): Le DataFrame contenant la colonne à traiter.
    column_name (str): Le nom de la colonne à traiter.
    top_n (int): Le nombre de valeurs les plus fréquentes à conserver (par défaut 5).
    others_label (str): L'étiquette à utiliser pour les valeurs regroupées (par défaut 'Autres').

    Returns:
    pandas.DataFrame: Une copie du DataFrame avec la colonne traitée.
    """
    # Créer une copie du DataFrame pour éviter de modifier l'original
    df_copy = df.copy()

    # Vérifier si la colonne existe
    if column_name not in df_copy.columns:
        raise ValueError(f"La colonne '{column_name}' n'existe pas dans le DataFrame.")

    # Obtenir les top_n valeurs les plus fréquentes (en excluant les valeurs nulles)
    value_counts = df_copy[column_name].value_counts(dropna=False)
    non_null_counts = value_counts[value_counts.index.notnull()]
    top_values = non_null_counts.nlargest(top_n).index.tolist()

    # Créer une fonction pour remplacer les valeurs
    def replace_value(value):
        if pd.isna(value):
            return value  # Garder les valeurs nulles telles quelles
        return value if value in top_values else others_label

    # Appliquer la fonction à la colonne
    df_copy[column_name] = df_copy[column_name].apply(replace_value)

    return df_copy

In [ ]:
data_tree.dtypes

In [ ]:
data_tree.columns

In [ ]:
columns = ['domanialite','genre', 'espece','stade_developpement']
for column in columns:
    if pd.api.types.is_object_dtype(data_tree[column]):
        data_tree = convert_to_top_5_and_others(data_tree, column)
data_tree.head()

In [ ]:
display_univariate_stats(data_tree)

In [ ]:
def table_values(df, column_name):
    effectifs = df[column_name].value_counts()
    modalites = effectifs.index # l'index des effectifs contient les modalités

    tab = pd.DataFrame(modalites, columns = [column_name]) # création du tableau à partir des modalités
    tab["n"] = effectifs.values
    tab["f"] = tab["n"] / len(df) # len(data_tree) renvoie la taille de l'échantillon
    return tab

In [ ]:
table_values(data_tree,'arrondissement')

Pour chacune des variables 'domnialite' 'genre' espece' on a 6 valeurs discrètes maintenant, mais pour la variable 'arrondissement' on a encore 25 valeurs discrètes, cependant au lieu d'utiliser directement la même approche de catégorisation, il est préférable de faire un petit traitement pour la représentation graphique: séparer les données de paris de la banlieue.

Pour cela on va répartir les valeurs de 'arrondissement' dans deux nouvelles variables: 'arrondissement_paris' 'arrondissement_banlieue'

In [ ]:
import numpy as np

# Liste des arrondissements de la banlieue
arrondissements_banlieue = ['SEINE-SAINT-DENIS', 'BOIS DE VINCENNES', 'VAL-DE-MARNE', 'HAUTS-DE-SEINE', 'BOIS DE BOULOGNE']

# Créer la colonne 'arrondissement_paris' : True si l'arrondissement n'est pas dans la banlieue, sinon NaN
data_tree['arrondissement_paris'] = np.where(data_tree['arrondissement'].isin(arrondissements_banlieue), np.nan, data_tree['arrondissement'])

# Créer la colonne 'arrondissement_banlieue' : True si l'arrondissement est dans la banlieue, sinon NaN
data_tree['arrondissement_banlieue'] = np.where(data_tree['arrondissement'].isin(arrondissements_banlieue), data_tree['arrondissement'], np.nan)
data_tree['arrondissement_paris_cat'] = data_tree['arrondissement_paris']
data_tree=convert_to_top_5_and_others(data_tree, "arrondissement_paris_cat")
display_univariate_stats(data_tree)

In [ ]:
data_tree.isnull().sum()

## Représentation des variables catégorielles sous forme de tableaux

In [ ]:
data_tree.columns

In [ ]:
table_values(data_tree,'domanialite')


In [ ]:
table_values(data_tree,'genre')


In [ ]:
table_values(data_tree,'espece')


In [ ]:
table_values(data_tree,'arrondissement')


In [ ]:
table_values(data_tree,'arrondissement_paris')


In [ ]:
table_values(data_tree,'arrondissement_paris_cat')


In [ ]:
table_values(data_tree,'arrondissement_banlieue')

In [ ]:
table_values(data_tree,'remarquable')

In [ ]:
data_tree.isnull().sum()

## Représentations graphiques

In [ ]:
data_tree.columns

On va commencer par représenter graphiquement chaque variable toute seule pour illustrer les distributions des valeurs sur celle_ci

### Variables discrètes

In [ ]:
#!pip install matplotlib

Domanialite:
variable catégorielle 'discrète'

In [ ]:
from matplotlib import pyplot as plt

# Get the counts for each category
category_counts = data_tree['domanialite'].value_counts()

# Plot pie chart
category_counts.plot.pie(autopct='%1.1f%%', figsize=(6, 6))

# Show the plot
plt.xlabel('Distribution des arbres sur les types de lieux')  # Optional: remove the default y-label
plt.ylabel('')
plt.show()

arrondissement: variable catégorielle 'discrète'

In [ ]:
# Get the counts for each category
category_counts = data_tree['arrondissement'].value_counts()

# Plot pie chart
category_counts.plot.pie(autopct='%1.1f%%', figsize=(6, 6))

# Show the plot
plt.xlabel('Distribution des arbres sur paris et la banlieue')  # Optional: remove the default y-label
plt.ylabel('')
plt.show()

In [ ]:
category_counts = data_tree['arrondissement'].value_counts()
# Plotting a bar chart
category_counts.plot.bar(color='skyblue', figsize=(8, 5))

# Customizing the plot
plt.title('Bar Chart Example')
plt.xlabel('Categories')
plt.ylabel('Values')
#plt.xticks(rotation=0)  # Rotate x-axis labels for better readability
plt.grid(axis='y')  # Optional: Add grid lines for better readability

# Show the plot
plt.show()

arrondissement_paris

In [ ]:
# Get the counts for each category
category_counts = data_tree['arrondissement_paris'].value_counts()

# Plot pie chart
category_counts.plot.pie(autopct='%1.1f%%', figsize=(6, 6))

# Show the plot
plt.xlabel('Distribution des arbres sur les arrondissements de paris')  # Optional: remove the default y-label
plt.ylabel('')
plt.show()

In [ ]:
category_counts = data_tree['arrondissement_paris'].value_counts()
# Plotting a bar chart
category_counts.plot.bar(color='skyblue', figsize=(8, 5))

# Customizing the plot
plt.title('Bar Chart Example')
plt.xlabel('Categories')
plt.ylabel('Values')
#plt.xticks(rotation=0)  # Rotate x-axis labels for better readability
plt.grid(axis='y')  # Optional: Add grid lines for better readability

# Show the plot
plt.show()

arrondissement_paris_cat

In [ ]:
# Get the counts for each category
category_counts = data_tree['arrondissement_paris_cat'].value_counts()

# Plot pie chart
category_counts.plot.pie(autopct='%1.1f%%', figsize=(6, 6))

# Show the plot
plt.xlabel('Distribution des arbres sur paris')  # Optional: remove the default y-label
plt.ylabel('')
plt.show()

arrondissement_banlieue

In [ ]:
# Get the counts for each category
category_counts = data_tree['arrondissement_banlieue'].value_counts()

# Plot pie chart
category_counts.plot.pie(autopct='%1.1f%%', figsize=(6, 6))

# Show the plot
plt.xlabel('Distribution des arbres sur la banlieue')  # Optional: remove the default y-label
plt.ylabel('')
plt.show()


genre: variable catégorielle 'discrète

In [ ]:
# Remplacer toutes les valeurs restantes (comme 'Non renseigné') qui n'ont pas été mappées
data_tree['genre'] = data_tree['genre'].fillna('Non renseigné')

# Get the counts for each category
category_counts = data_tree['genre'].value_counts()

# Plot pie chart
category_counts.plot.pie(autopct='%1.1f%%', figsize=(6, 6))

# Show the plot
plt.xlabel("Diffèrents genre d\'arbres")  # Optional: remove the default y-label
plt.ylabel('')
plt.show()

espece: variable catégorielle 'discrète'

In [ ]:
# Remplacer toutes les valeurs restantes (comme 'Non renseigné') qui n'ont pas été mappées
data_tree['espece'] = data_tree['espece'].fillna('Non renseigné')


category_counts = data_tree['espece'].value_counts()

# Plot pie chart
category_counts.plot.pie(autopct='%1.1f%%', figsize=(6, 6))

# Show the plot
plt.xlabel('Différentes espèces d\'arbres')  # Optional: remove the default y-label
plt.ylabel('')
plt.show()

In [ ]:
# Remplacer toutes les valeurs restantes (comme 'Non renseigné') qui n'ont pas été mappées
data_tree['stade_developpement'] = data_tree['stade_developpement'].fillna('Non renseigné')

# Obtenir le comptage des catégories ('non', 'oui' et 'Non renseigné')
category_counts = data_tree['stade_developpement'].value_counts()

# Afficher les résultats
print(category_counts)

stade_developpement: variable catégorielle 'discrète'

In [ ]:
# Get the counts for each category
category_counts = data_tree['stade_developpement'].value_counts()

# Plot pie chart
category_counts.plot.pie(autopct='%1.1f%%', figsize=(6, 6))

# Show the plot
plt.xlabel('Différentes des périodes d\'âges d\'arbres')  # Optional: remove the default y-label
plt.ylabel('')
plt.show()

remarquable: malgres que cette variable contient des valeurs numériques, elles sont discrètes, donc la même représentation graphiques que les précedentes variables.

In [ ]:
data_tree['remarquable_cat'] = data_tree['remarquable'].map({0: 'non', 1: 'oui'})

# Remplacer toutes les valeurs restantes (comme 'Non renseigné') qui n'ont pas été mappées
data_tree['remarquable_cat'] = data_tree['remarquable_cat'].fillna('Non renseigné')

# Obtenir le comptage des catégories ('non', 'oui' et 'Non renseigné')
category_counts = data_tree['remarquable_cat'].value_counts()

# Afficher les résultats
print(category_counts)


In [ ]:
# Plot pie chart
category_counts.plot.pie(autopct='%1.1f%%', figsize=(6, 6))

# Show the plot
plt.xlabel('Les arbres selon leur remarquabilité ou non')  # Optional: remove the default y-label
plt.ylabel('')
plt.show()

### Variables continues

circonference_cm:

In [ ]:
# Histogramme
# Plotting a histogram for the 'Values' column
data_tree['circonference_cm'].plot.hist(bins=10, color='skyblue', alpha=0.7, edgecolor='black', figsize=(8, 5))

# Customizing the plot
plt.title('Histogram des circonférences')
plt.xlabel('Circonférence (cm)')
plt.ylabel('Fréquence')
plt.grid(axis='y')  # Optional: Add grid lines for better readability

# Show the plot
plt.show()


In [ ]:
# boite a moustache
data_tree.boxplot(column="circonference_cm", vert=False)
plt.show()

In [ ]:
data_tree['circonference_cm'].sort_values(ascending=False).head(30)


hauteur_cm:

In [ ]:
# Histogramme
# Plotting a histogram for the 'Values' column
data_tree['hauteur_cm'].plot.hist(bins=10, color='skyblue', alpha=0.7, edgecolor='black',figsize=(8, 5))

# Customizing the plot
plt.title('Histogram des hauteurs')
plt.xlabel('Hauteur (cm)')
plt.xlim(0, data_tree['hauteur_cm'].max())
plt.xticks(rotation=0)  # Rotate x-axis labels for better readability
plt.ylabel('Fréquence')
plt.grid(axis='y')  # Optional: Add grid lines for better readability

# Show the plot
plt.show()

In [ ]:
data_tree.boxplot(column="hauteur_cm", vert=False)
plt.show()

In [ ]:
data_tree['hauteur_cm'].sort_values(ascending=False).head(30)

### Représentation cartogarphiques des coordonnées

geo_point_2d_a  et geo_point_2d_b sont des données cartographiques, ça ne sert à rien d'afficher les distributions de valeurs des ces denières, par contre on va proposer un autre type d'illustrations: "une carte !!!!"


Pour cela, la bibliothèque dont on a besoin s'appelle folium

In [ ]:
!pip install folium

In [ ]:
import folium

# Créer une carte centrée sur la moyenne des coordonnées
center_lat = data_tree['geo_point_2d_a'].mean()
center_lon = data_tree['geo_point_2d_b'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12 ,width=400, height=300)

# Ajouter un marqueur pour chaque arbre
for idx, row in data_tree.iterrows():
    folium.Marker(
        location=[row['geo_point_2d_a'], row['geo_point_2d_b']],
        popup=f"Arbre {idx}"
    ).add_to(m)

#m.save("carte_arbres.html")
# Afficher la carte
m

# Fin